In [ ]:
import imagej 

#state where your fiji.app is located to access plugins and extensions

ij_path = 'D:/Fiji.app'
ij = imagej.init(ij_path, mode='interactive', add_legacy=True) 

print(ij.getVersion())

import scyjava as sj
import pandas as pd
import sys, glob
import os
import csv
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import seaborn as sns
import glob
#from natsort import index_natsorted
import warnings
import random

gui = sj.jimport('ij.gui.YesNoCancelDialog')
Frame = sj.jimport('java.awt.Frame') 
File = sj.jimport('java.io.File') 
Color= sj.jimport('java.awt.Color') 


IJ = sj.jimport('ij.IJ')
ImageJ= sj.jimport('ij.ImageJ')
Imagestack= sj.jimport('ij.ImageStack')
w = sj.jimport('ij.WindowManager')
 
ImageCalculator = sj.jimport('ij.plugin.ImageCalculator')
FolderOpener = sj.jimport('ij.plugin.FolderOpener')
ZProjector = sj.jimport('ij.plugin.ZProjector')
 
Model =  sj.jimport('fiji.plugin.trackmate.Model')
Settings= sj.jimport('fiji.plugin.trackmate.Settings')
TrackMate = sj.jimport('fiji.plugin.trackmate.TrackMate')
Logger= sj.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= sj.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= sj.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= sj.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
LogRecorder = sj.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= sj.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = sj.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = sj.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = sj.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = sj.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = sj.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = sj.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = sj.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = sj.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= sj.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')



class image_processor():
    #image_directory accepts a path item with either nontransformed images (stack_reg=True) or transformed images(False)
    #save_path a folder or directory where you want to save all processed images and files
    #can switch the type of image being processed for stack_reg (asn't be tested on czi yet)
    def __init__(self, image_directory, save_path, stack_reg=True, image_type='tif'):
        self.image_directory = image_directory
        self.save_path = save_path
        self.stack_reg = stack_reg
        self.image_type=image_type
        
    #takes the save_path and creates four new folders that will be used to save csv, overlays, and xmls to
    #if self.stack_reg=True, create a directory to save the images post-processing
    #Returns the path of directories to save the experiments data to
    def save_directories(self):
        dir_list = []
        save_list = ['csv_files/', 'overlay_files/', 'xml_files/']
        if self.stack_reg == True: 
            save_list.append('stack_dir/')
        for direct in save_list:
            save_dir = os.path.join(self.save_path, direct)
            doesExist = os.path.exists(save_dir)
            dir_list.append(save_dir)
            if doesExist == False:
                os.mkdir(save_dir)
        return dir_list
    
    #Returns a list of all image files inside the directory
    def get_image_list(self):
        path_to_images = []
        for directory, dir_names, file_names in os.walk(self.image_directory):
            for file_name in file_names:
                if file_name.split('.')[-1] == self.image_type:
                    full_path = os.path.normpath(os.path.join(directory, file_name))
                    path_to_images.append(full_path)
            res = [i for n, i in enumerate(path_to_images) if i not in path_to_images[:n]]
            path_to_images = res
            return path_to_images	
    
    #Operates if the self.stack_reg=True
    #utilizes the legacy version of stack reg
    #Works on a multidimension 4d image: can be used on a 3d, but a try or if implementation  
    #needs to be added if there isless than 1 frame. 
    #Saves this to the stack_reg directory created before 
    def stack_reg_images(self):
        if self.stack_reg == True:
            save_location = self.save_directories()[3]
            path_to_images = self.get_image_list()
            img_list = []
            for i in range(len(path_to_images)):
                imp  = ij.IJ.openImage(path_to_images[i])
                imp.show()
                other_title = str(imp.getTitle())
                img_title = path_to_images[i].split('\\')[-1].split('.')[0]
                print(img_title)
                img_dim = imp.getDimensions()[4]
                for frame in range(img_dim):
                    frame=frame+1
                    w.getImage(other_title)
                    if frame == 1:
                        IJ.run("Duplicate...", "title=Stack_reg_image duplicate frames="+str(frame))
                        IJ.run("StackReg ", "transformation=[Rigid Body]")
                    elif frame > 1:
                        IJ.selectWindow(other_title)
                        IJ.run("Duplicate...", "title=channels duplicate frames="+str(frame))
                        IJ.run("StackReg ", "transformation=[Rigid Body]")
                        IJ.run("Concatenate...", "title=Stack_reg_image open image1=Stack_reg_image image2=channels image3=[-- None --]")
                img_stackreg =  w.getImage("Stack_reg_image")
                img_sr_title = str(img_stackreg.getTitle())
                stack_reg_image_file = save_location+img_title+"_StackRegImage.tif"
                IJ.save(img_stackreg, stack_reg_image_file)
                img_list.append(stack_reg_image_file)
                img_stackreg.close()
                imp.close()
        return img_list
    
    
    #Cellpose-Trackmate processor takes the images from stack_reg_dir if self.stack_reg =True or from
    #self.image_directory if stack_reg = False
    #imaging dates can be either a string or list of strings
    #if construct_in_file is False, the cell column will have a none data type
    #if construct_in_file is set to true, set the position the construct is located in (assuming the split is '_')
    #crossexcitation and blled through values for the scope or set up can be set
    #cellpose_exe is dependent on the set up env, you should have a python.exe in the cellpose env
    #model_location also depend on setup and which model you choose. 
    #model has to be set to Pretrained.Model, where Model can be CYTO, CYTO2, NUCLEI, or CUSTOM
    #if set to custom ensure the cellpose_model_location is defined appropiately
    #return value is a Pandas Dataframe of all the images and constructs in the directory
    #save a capture of the overlay, csv files, and xml file of the tracks
    def cellpose_processing(self, 
                            imaging_dates,
                            construct_in_file=False,
                            construct=None,
                            experiment_in_file = False,
                            experiment =None,
                            well_in_file=False,
                            well=None,
                            position_in_file=False,
                            position=None,
                            cx=0.19,
                            bt=0.53,
                            cellpose_exe='C:/Users/Shahar_group_scope/anaconda3/envs/cellpose/python.exe', 
                            cellpose_model=PretrainedModel.CYTO, 
                            cellpose_model_location='C:/Users/Shahar_group_scope/Documents/Patrick/Stable_cells/24hr_troubleshooting/20230414_GS32/model_creation/models/CP_20230420_074409'):
        
        if self.stack_reg == True:
            path_to_images = self.stack_reg_images()
        else:
            path_to_images = self.get_image_list()
        csv_save=            self.save_directories()[0]
        overlay_save=        self.save_directories()[1]
        xml_save=            self.save_directories()[2]
        full_df = pd.DataFrame()
        for i in range(len(path_to_images)):
            imp  = ij.IJ.openImage(path_to_images[i])
            img_title = str(imp.getTitle())
            
            if isinstance(imaging_dates, list):
                imaging_date = imaging_dates[i]
            else:
                imaging_date = imaging_dates
            if construct_in_file == True: 
                cell = img_title.split("_")[construct]
            else:
                cell= 0
            if well_in_file == True: 
                well = img_title.split("_")[well]
            else:
                well = 1
            if well_in_file == True: 
                posi = img_title.split("_")[position]
            else:
                posi = 1 
                
            process_date = date.today()
            
            csv_path = os.path.join(csv_save, img_title.split('.')[0])
            overlay_path = os.path.join(overlay_save, img_title.split('.')[0])
            xml_path = xml_save
        
            model = Model()
            logger = LogRecorder( Logger.VOID_LOGGER )
        
            model.setLogger(Logger.IJ_LOGGER)

            settings = Settings(imp)
            
            settings.detectorFactory = CellposeDetectorFactory()
            settings.detectorSettings = {
                'TARGET_CHANNEL' : ij.py.to_java(1),
                'OPTIONAL_CHANNEL_2' : ij.py.to_java(0),
                'CELLPOSE_PYTHON_FILEPATH' : cellpose_exe,
                'CELLPOSE_MODEL' : cellpose_model,	    
                'CELLPOSE_MODEL_FILEPATH' : cellpose_model_location,
                'CELL_DIAMETER' : 0.0,
                'USE_GPU' : True,
                'SIMPLIFY_CONTOURS' : False,
            }
            
           
            #tracker configuration, Variables can be added or subtracted, numbers can be changed accordingly
            settings.trackerFactory = SparseLAPTrackerFactory()
            settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
            settings.trackerSettings[ 'LINKING_MAX_DISTANCE' ]= 25.0
            settings.trackerSettings[ 'GAP_CLOSING_MAX_DISTANCE' ]= 25.0
            settings.trackerSettings[ 'MAX_FRAME_GAP' ]= ij.py.to_java(3)
           
            settings.addAllAnalyzers()
           
           
            settings.initialSpotFilterValue = 1.
            
            print(str(settings))
           
            trackmate = TrackMate(model, settings)
            trackmate.computeSpotFeatures( True )
            trackmate.computeTrackFeatures( True )
            trackmate.getModel().setLogger( logger )
            
            ok = trackmate.checkInput()
            if not ok:
                sys.exit(str(trackmate.getErrorMessage()))
            
            ok = trackmate.process()
            if not ok:
                sys.exit(str(trackmate.getErrorMessage()))
           
            saveFile = TMUtils.proposeTrackMateSaveFile( settings, logger )
            model.getLogger().log('Found ' + str(model.getTrackModel().nTracks(True)) + ' tracks.')            
           
            sm = SelectionModel( model )            
           
            ds = DisplaySettingsIO.readUserDefault()
                        
            displayer =  HyperStackDisplayer( model, sm, imp, ds ) 
            displayer.render()
           
            image = trackmate.getSettings().imp
            capture = CaptureOverlayAction.capture(image, -1, imp.getNFrames(), logger)
            capture.setTitle(img_title+"_TracksOverlay")
            cap_title = capture.getTitle()
            IJ.save(capture, overlay_path+'_TracksOverlay.avi')
            #IJ.run(capture, "Calibration Bar...", "location=[Upper Right] fill=White label=Black number=5 decimal=0 font=12 zoom=1 overlay")
            
            outFile = File(xml_path, img_title.split('.')[0]+"_exportTracks.xml")   #this will write the tracks only XML
            ExportTracksToXML.export(model, settings, outFile)
            outFile = File(xml_path, img_title.split('.')[0]+"_exportModel.xml")  # this will write the full trackmate xml.
            xmlwriter = TmXmlWriter(outFile) 
            xmlwriter.appendModel(model)
            xmlwriter.appendSettings(settings)
            xmlwriter.writeToFile()                                             
              
            fm = model.getFeatureModel()
            image_df=pd.DataFrame()
                      
            for id in model.getTrackModel().trackIDs(True):               
                track_df = pd.DataFrame()
                track_id = ij.py.from_java(id)
                track = model.getTrackModel().trackSpots(id)
                for spot in track:
                    sid = ij.py.from_java(spot.ID())        
                    t=ij.py.from_java(spot.getFeature('FRAME'))
                    q=ij.py.from_java(spot.getFeature('QUALITY'))
                    r=ij.py.from_java(spot.getFeature('RADIUS'))
                    a=ij.py.from_java(spot.getFeature('AREA'))
                    p=ij.py.from_java(spot.getFeature('PERIMETER'))
                    c=ij.py.from_java(spot.getFeature('CIRCULARITY'))
                    s=ij.py.from_java(spot.getFeature('SOLIDITY'))
                    mean_1=ij.py.from_java(spot.getFeature('MEAN_INTENSITY_CH1'))
                    mean_2=ij.py.from_java(spot.getFeature('MEAN_INTENSITY_CH2'))
                    mean_3=ij.py.from_java(spot.getFeature('MEAN_INTENSITY_CH3'))
                    std_1=ij.py.from_java(spot.getFeature('STD_INTENSITY_CH1'))
                    std_2=ij.py.from_java(spot.getFeature('STD_INTENSITY_CH2'))
                    std_3=ij.py.from_java(spot.getFeature('STD_INTENSITY_CH3'))
                    mean_2_corrected= mean_2-mean_1*bt-mean_3*cx
                    Efret= mean_2_corrected/(mean_1+mean_2_corrected)
                    df = pd.DataFrame({'date_processed':[process_date], 
                                       'date_imaged':[imaging_date],
                                       'file_name': [img_title.split('.')[0]],
                                       'model_used':[cellpose_model], 
                                       'model_location':[cellpose_model_location], 
                                       'construct':[cell],
                                       'experiment': experiment,
                                       'well': well,
                                       'position': posi,
                                       'track_id':[track_id], 
                                       'frame': [t], 
                                       'Spot_ID':[sid], 
                                       'quality':[q],
                                       'radius':[r], 
                                       'area':[a], 
                                       'perimeter':[p], 
                                       'circularity': [c],
                                       'solidity': [s], 
                                       'donor_mean':[mean_1],
                                       'acceptor_mean': [mean_2], 
                                       'acceptor_mean_corrected': [mean_2_corrected], 
                                       'efret':[Efret], 
                                       'direct_acceptor_mean':[mean_3],
                                       'donor_Std': std_1,
                                       'acceptor_std_ch2': std_2, 
                                       'direct_acceptor_std': std_3,})
                    
                    track_df = pd.concat([track_df, df])
                    
                image_df = pd.concat([image_df, track_df])        
            image_df.to_csv(csv_path+'.csv', index=False)   
            imp.close()
            full_df = pd.concat([full_df, image_df])
        full_df.to_csv(csv_save+'combined_data.csv', index=False)
        return full_df

In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import sys

In [2]:
scyjava.config.add_option('-Xmx60g')
start_dir = os.getcwd()
#ij = imagej.init('sc.fiji:fiji', mode='interactive')
ij = imagej.init('/Users/volkenlab/Documents/Jacques_Data/Fiji.app/', mode='interactive')
## Something about this init() function changes the current working directory.
#ij.getApp().getInfo(True)
ij.ui().showUI()
os.chdir(start_dir)
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version n

'2.14.0/1.54f'

{TARGET_CHANNEL=1, INTENSITY_THRESHOLD=5.0, SIMPLIFY_CONTOURS=false}
{TARGET_CHANNEL=1, INTENSITY_THRESHOLD=5.0, SIMPLIFY_CONTOURS=false}
{INTENSITY_THRESHOLD=5.0, SIMPLIFY_CONTOURS=false}
{TARGET_CHANNEL=1, INTENSITY_THRESHOLD=5.0, SIMPLIFY_CONTOURS=false}
{TARGET_CHANNEL=(int)1, INTENSITY_THRESHOLD=5.0, SIMPLIFY_CONTOURS=false}
{TARGET_CHANNEL=(int)1, INTENSITY_THRESHOLD=5.0, SIMPLIFY_CONTOURS=false}
{TARGET_CHANNEL=1, INTENSITY_THRESHOLD=5.0, SIMPLIFY_CONTOURS=false}
{TARGET_CHANNEL=1, INTENSITY_THRESHOLD=5.0, SIMPLIFY_CONTOURS=false}
{TARGET_CHANNEL=1, INTENSITY_THRESHOLD=5.0, SIMPLIFY_CONTOURS=false}


In [19]:
import sys
import os

Model =  scyjava.jimport('fiji.plugin.trackmate.Model')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')

#reload(sys)
#sys.setdefaultencoding("utf-8")

# Directory
# Here, the user can specify the directory where the cell images are located
src = "/Users/volkenlab/Documents/Anushka/Images/"
out = src+"Output/"
if not os.path.exists(out):
    os.makedirs(out)

# Parameters for Detection
# Here, the user can specify parameters for the detection step in Trackmate (Threshold Detector)
dsettings = {
    'TARGET_CHANNEL' : ij.py.to_java(1),
    'SIMPLIFY_CONTOURS' : False,
    'INTENSITY_THRESHOLD' : 5.0,
}

# Parameters for Tracking
# Here, the user can specify parameters for the tracking step in Trackmate (LAP Tracker)
frame_gap = 2
tsettings = {
    'LINKING_MAX_DISTANCE' : 15.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 15.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
}

for image in os.listdir(src):
    if (image[len(image)-4:] == ".tif"):

        # Open Image
        imp = ij.IJ.openImage(src + image)
        imp.show()
        
        # Create Model
        model = Model()
        settings = Settings(imp)
        
        # Detector
        settings.detectorFactory = ThresholdDetectorFactory()
        for parameter, value in dsettings.items():
            #settings.detectorSettings.update({parameter:value})
            settings.detectorSettings[parameter] = value
        filter1 = FeatureFilter('QUALITY', 198, True)
        settings.addSpotFilter(filter1)
        print(settings.detectorSettings)
        
        # Tracker
        settings.trackerFactory = SparseLAPTrackerFactory()
        settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
        for parameter, value in tsettings.items():
            #settings.trackerSettings.update({parameter:value})
            settings.trackerSettings[parameter] = value
        
        # Execute Tracking
        trackmate = TrackMate(model, settings)
        ok = trackmate.checkInput()
        if not ok:
            sys.exit(str(trackmate.getErrorMessage()))
        ok = trackmate.process()
        if not ok:
            sys.exit(str(trackmate.getErrorMessage()))
        selectionModel = SelectionModel(model)
        
        # Display
        ds = DisplaySettingsIO.readUserDefault()
        displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
        displayer.render()
        displayer.refresh()
        trackscheme = TrackScheme(model, selectionModel, ds)
        trackscheme.render()
        
        # Save Data
        outFile = Path(out+image+"_exportModel.xml")
        writer = TmXmlWriter(outFile)
        writer.appendModel(model)
        writer.appendSettings(settings)
        writer.writeToFile()
        csvFileTracks = Path(out+image+"_exportTracks.csv")
        trackTableView = TrackTableView(model, selectionModel, ds)
        trackTableView.getSpotTable().exportToCsv(csvFileTracks)

[java.lang.Enum.toString] Starting detection process using 20 threads.
[java.lang.Enum.toString] Detection processes 20 frames simultaneously and allocates 1 thread per frame.
[java.lang.Enum.toString] Detection...[java.lang.Enum.toString] 
[java.lang.Enum.toString] Found 75126 spots.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Starting initial filtering process.
[java.lang.Enum.toString] Computing spot features over 20 frames simultaneously and allocating 1 thread per frame.
[java.lang.Enum.toString] Calculating 75126 spots features...[java.lang.Enum.toString] 
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Computation done in 32 ms.
[java.lang.Enum.toString] Starting spot filtering process.
[java.lang.Enum.toString] Starting tracking process.
[java.lang.Enum.toString] Frame to frame linking...[java.lang.Enum.toString] 
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Creating the segment linking cost matrix...[java.lang.Enum.toString] 
[java.lang.Enum.toString] Cre